In [ ]:
#bruke vår egen scraper i stedet for
import lyricsgenius

def get_lyrics(artist_name, max_songs = float('inf')):
    genius = lyricsgenius.Genius('l8-nnpE9RfPvXrIpXq1zhiO1SOdcwCD5m3sOUw9rZlrtWRY1f-9K4wD5tdWK2xN6')
    genius.timeout = 15
    artist = genius.search_artist(artist_name, max_songs=10, sort='popularity')
    print(artist)
    songs = []
    for song in artist.songs:
        songs.append(song.lyrics)
    return songs

def get_lyrics_and_save(artist_name, max_songs = float('inf'))
    songs = get_lyrics(artist_name, max_songs)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(lyrics_list,test_size=0.15)

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


NameError: name 'lyrics_list' is not defined

In [5]:
with open(f'train{artist_name}.txt', 'w') as f:
  for t in train:
    f.write(t)
    f.write(' ')


with open(f'test{artist_name}.txt', 'w') as f:
  for t in test:
    f.write(t)
    f.write(' ')


In [4]:
artist_name = 'drake'
train_path = f'train{artist_name}.txt'
test_path = f'test{artist_name}.txt'

In [5]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

2023-04-24 00:22:47.124972: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (335558 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
from transformers import AutoModelWithLMHead, TrainingArguments, Trainer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)

training_args = TrainingArguments(
    output_dir=f"./gpt2-{artist_name}-2", 
    overwrite_output_dir=True,
    num_train_epochs=50, 
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    eval_steps = 100, 
    save_steps=800, 
    warmup_steps=500
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [9]:
trainer.train()


/usr/local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4100 [00:00<?, ?it/s]

{'loss': 3.4223, 'learning_rate': 5e-05, 'epoch': 6.1}


In [1]:
trainer.save_model()

NameError: name 'trainer' is not defined

In [28]:
from transformers import pipeline

diomedes = pipeline('text-generation',model=f'./gpt2-{artist_name}-2', tokenizer='gpt2')

results = diomedes('Italia')[0]['generated_text']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
